In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, Normalizer
import pickle
from sklearn.model_selection import train_test_split
import keras

In [145]:
%matplotlib inline

In [146]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train.shape, test.shape

((14364, 55), (756, 54))

In [147]:
for i in train.keys():
    if not i in test.keys():
        print(i)

Cover_Type


In [148]:
y = train['Cover_Type']
del train['Cover_Type']

In [149]:
def data_prepro(data, s=None):
    
    if s is None:
        s = StandardScaler()
        s.fit(data)
        pickle.dump( s, open('s.pkl', 'wb') )

    data = s.transform(data)
    
    """
    cols=[
        'Horizontal_Distance_To_Hydrology',
        'Vertical_Distance_To_Hydrology',
        'Horizontal_Distance_To_Roadways',
        'Hillshade_9am',
        'Hillshade_Noon',
        'Hillshade_3pm',
        'Horizontal_Distance_To_Fire_Points',
    ]
    # to negtive
    for c in cols:
        #_ = plt.hist(data[c]); plt.show()
        if ('Horizontal' in c) or ('Vertical' in c):
            data[c] = data[c].apply(lambda x:-x)
        data[c] = np.log(data[c])
        #_ = plt.hist(data[c]); plt.show()
    """
    
    data = np.nan_to_num(data)
    return data

In [150]:
data = np.concatenate( (train, test), 0 )
data.shape

(15120, 54)

In [151]:
data_1 = data_prepro(data)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [152]:
data_train = data_1[:train.shape[0], :]
data_test = data_1[train.shape[0]:, :]
data_train.shape

(14364, 54)

In [153]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [16]:
cls = GradientBoostingClassifier()
cls.fit(data_train,y )
cls.score(data_train,y)

0.8373015873015873

In [17]:
cls = RandomForestClassifier()
cls.fit(data_train,y )
cls.score(data_train,y)

0.9946393762183235

In [46]:
df = pd.DataFrame( {'id': test.index, 'class':cls.predict(data_test) }, index=range(len(data_test)))
df.to_csv('sub_4.csv', index=False)

In [47]:
ans_3 = pd.read_csv('sub_3.csv')
ans_4 = pd.read_csv('sub_4.csv')

In [48]:
(ans_3['class']  == ans_4['class']).all()

False

In [184]:
from keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU

In [185]:
model = keras.models.Sequential()

In [186]:

model.add(Dense(units=400,                    #建立輸入層至隱藏層連接
           input_dim=54,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=200,                    #建立輸入層至隱藏層連接
           input_dim=400,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=100,                    #建立輸入層至隱藏層連接
           input_dim=200,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dense(units=50,                    #建立輸入層至隱藏層連接
           input_dim=100,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=20,                    #建立輸入層至隱藏層連接
           input_dim=50,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=7,                    #建立輸入層至隱藏層連接
           input_dim=20,                      #輸入神經元數目=784
           kernel_initializer='normal',  #以常態分佈亂數初始化參數
           activation='softmax')) 

/opt/conda/lib/python3.6/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [187]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 400)               22000     
_________________________________________________________________
dropout_13 (Dropout)         (None, 400)               0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 400)               1600      
_________________________________________________________________
dense_47 (Dense)             (None, 200)               80200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 200)               0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 200)               800       
_________________________________________________________________
dense_48 (Dense)             (None, 100)               20100     
__________

In [188]:
y_ = y-1

In [189]:
from keras.utils import to_categorical

In [190]:
y_train = to_categorical(y_)

In [191]:
train_train, train_test = train_test_split(train, )

In [192]:
from keras import optimizers

In [193]:
model.compile(loss='categorical_crossentropy',   #設定損失函數
            optimizer=optimizers.Adam(lr=3e-3),                     #設定最佳化方法
            metrics=['accuracy']) 

In [194]:
train_history=model.fit(x=data_train,    #正規化的訓練樣本圖片
                        y=y_train,          #One-hot 編碼
                        validation_split=0.2,      #分出 20% 做驗證
                        epochs=50,                      #設定訓練週期 (輪)
                        batch_size=256,              #每批次訓練筆數
                        verbose=2) 

Train on 11491 samples, validate on 2873 samples
Epoch 1/50
 - 3s - loss: 1.3761 - acc: 0.5349 - val_loss: 0.8672 - val_acc: 0.6572
Epoch 2/50
 - 0s - loss: 0.9143 - acc: 0.6455 - val_loss: 0.7373 - val_acc: 0.6982
Epoch 3/50
 - 0s - loss: 0.8056 - acc: 0.6766 - val_loss: 0.6921 - val_acc: 0.7104
Epoch 4/50
 - 0s - loss: 0.7402 - acc: 0.7018 - val_loss: 0.6465 - val_acc: 0.7389
Epoch 5/50
 - 0s - loss: 0.7137 - acc: 0.7102 - val_loss: 0.6493 - val_acc: 0.7254
Epoch 6/50
 - 0s - loss: 0.6871 - acc: 0.7190 - val_loss: 0.6135 - val_acc: 0.7327
Epoch 7/50
 - 0s - loss: 0.6553 - acc: 0.7339 - val_loss: 0.5745 - val_acc: 0.7630
Epoch 8/50
 - 0s - loss: 0.6355 - acc: 0.7416 - val_loss: 0.5779 - val_acc: 0.7633
Epoch 9/50
 - 0s - loss: 0.6200 - acc: 0.7474 - val_loss: 0.5713 - val_acc: 0.7671
Epoch 10/50
 - 0s - loss: 0.6055 - acc: 0.7592 - val_loss: 0.5479 - val_acc: 0.7703
Epoch 11/50
 - 0s - loss: 0.5955 - acc: 0.7575 - val_loss: 0.5487 - val_acc: 0.7762
Epoch 12/50
 - 0s - loss: 0.5892 - a

In [195]:
model.compile(loss='categorical_crossentropy',   #設定損失函數
            optimizer=optimizers.Adam(lr=3e-4),                     #設定最佳化方法
            metrics=['accuracy'])
train_history=model.fit(x=data_train,    #正規化的訓練樣本圖片
                        y=y_train,          #One-hot 編碼
                        validation_split=0.2,      #分出 20% 做驗證
                        epochs=50,                      #設定訓練週期 (輪)
                        batch_size=256,              #每批次訓練筆數
                        verbose=2)

Train on 11491 samples, validate on 2873 samples
Epoch 1/50
 - 3s - loss: 0.3532 - acc: 0.8608 - val_loss: 0.4295 - val_acc: 0.8402
Epoch 2/50
 - 0s - loss: 0.3453 - acc: 0.8663 - val_loss: 0.4214 - val_acc: 0.8434
Epoch 3/50
 - 0s - loss: 0.3344 - acc: 0.8678 - val_loss: 0.4196 - val_acc: 0.8413
Epoch 4/50
 - 0s - loss: 0.3345 - acc: 0.8644 - val_loss: 0.4189 - val_acc: 0.8413
Epoch 5/50
 - 0s - loss: 0.3244 - acc: 0.8748 - val_loss: 0.4165 - val_acc: 0.8416
Epoch 6/50
 - 0s - loss: 0.3203 - acc: 0.8724 - val_loss: 0.4178 - val_acc: 0.8437
Epoch 7/50
 - 0s - loss: 0.3248 - acc: 0.8714 - val_loss: 0.4194 - val_acc: 0.8451
Epoch 8/50
 - 0s - loss: 0.3246 - acc: 0.8720 - val_loss: 0.4173 - val_acc: 0.8437
Epoch 9/50
 - 0s - loss: 0.3193 - acc: 0.8756 - val_loss: 0.4172 - val_acc: 0.8420
Epoch 10/50
 - 0s - loss: 0.3297 - acc: 0.8682 - val_loss: 0.4180 - val_acc: 0.8420
Epoch 11/50
 - 0s - loss: 0.3149 - acc: 0.8785 - val_loss: 0.4156 - val_acc: 0.8448
Epoch 12/50
 - 0s - loss: 0.3123 - a

In [202]:
model.compile(loss='categorical_crossentropy',   #設定損失函數
            optimizer=optimizers.Adam(lr=1e-5),                     #設定最佳化方法
            metrics=['accuracy'])
train_history=model.fit(x=data_train,    #正規化的訓練樣本圖片
                        y=y_train,          #One-hot 編碼
                        validation_split=0.2,      #分出 20% 做驗證
                        epochs=50,                      #設定訓練週期 (輪)
                        batch_size=256,              #每批次訓練筆數
                        verbose=2)

Train on 11491 samples, validate on 2873 samples
Epoch 1/50
 - 4s - loss: 0.2481 - acc: 0.8985 - val_loss: 0.4244 - val_acc: 0.8545
Epoch 2/50
 - 0s - loss: 0.2486 - acc: 0.9008 - val_loss: 0.4245 - val_acc: 0.8556
Epoch 3/50
 - 0s - loss: 0.2546 - acc: 0.9011 - val_loss: 0.4246 - val_acc: 0.8559
Epoch 4/50
 - 0s - loss: 0.2551 - acc: 0.9004 - val_loss: 0.4244 - val_acc: 0.8559
Epoch 5/50
 - 0s - loss: 0.2528 - acc: 0.8983 - val_loss: 0.4251 - val_acc: 0.8559
Epoch 6/50
 - 0s - loss: 0.2457 - acc: 0.9011 - val_loss: 0.4249 - val_acc: 0.8559
Epoch 7/50
 - 0s - loss: 0.2465 - acc: 0.9049 - val_loss: 0.4249 - val_acc: 0.8559
Epoch 8/50
 - 0s - loss: 0.2487 - acc: 0.9036 - val_loss: 0.4247 - val_acc: 0.8559
Epoch 9/50
 - 0s - loss: 0.2509 - acc: 0.9006 - val_loss: 0.4246 - val_acc: 0.8562
Epoch 10/50
 - 0s - loss: 0.2536 - acc: 0.9021 - val_loss: 0.4248 - val_acc: 0.8566
Epoch 11/50
 - 0s - loss: 0.2495 - acc: 0.9027 - val_loss: 0.4251 - val_acc: 0.8562
Epoch 12/50
 - 0s - loss: 0.2521 - a

In [203]:
model.save_weights('model_20190830.h5')

In [204]:
pred = np.argmax( model.predict(data_test), 1)+1

In [207]:
df = pd.DataFrame( {'id': test.index, 'class':pred}, index=range(len(data_test)))
df.to_csv('sub_6.csv', index=False)

In [206]:
data_test.shape

(756, 54)